# Training Machine Learning

(10/27/2022) not final virsion, it will be modified later 


## 1. Import and Data Load

In [1]:
# 1. Data Loadimport numpy as np
import pandas as pd
import os
import math
import numpy as np

from scipy import stats #Analysis x
from scipy.stats import norm 

#import missingno as msno
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
 
from sklearn.preprocessing import LabelEncoder # for preprocessing

C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = "./Dataset/"

# original data -> data => dataset
data_01 = pd.read_csv(path+"Dataset01/gesture01.csv")
data_02 = pd.read_csv(path+"Dataset02/gesture02.csv")
data_03 = pd.read_csv(path+"Dataset03/gesture03.csv")
data_04 = pd.read_csv(path+"Dataset04/gesture04.csv")
data_05 = pd.read_csv(path+"Dataset05/gesture05.csv")
data_06 = pd.read_csv(path+"Dataset06/gesture06.csv")
data_07 = pd.read_csv(path+"Dataset07/gesture07.csv")
data_08 = pd.read_csv(path+"Dataset08/gesture08.csv")
dataset = [data_01, data_02, data_03, data_04,
           data_05, data_06, data_07, data_08]

data_08

,Unnamed: 0,0,1,2,3,4,0.1,1.1,2.1
0,0,HandLandmark.WRIST,Right,0.449992,0.404358,0.000001,0.000000,0.000000,0.000000
1,1,HandLandmark.THUMB_CMC,Right,0.393602,0.469066,-0.053380,-0.056390,0.064708,-0.053381
2,2,HandLandmark.THUMB_MCP,Right,0.362708,0.537984,-0.099210,-0.087284,0.133626,-0.099211
3,3,HandLandmark.THUMB_IP,Right,0.372486,0.603303,-0.130151,-0.077506,0.198945,-0.130152
4,4,HandLandmark.THUMB_TIP,Right,0.379628,0.661298,-0.161920,-0.070363,0.256940,-0.161921
...,...,...,...,...,...,...,...,...,...
6295,6295,HandLandmark.RING_FINGER_TIP,Left,0.422629,0.776004,-0.141457,-0.118355,0.631340,-0.141457
6296,6296,HandLandmark.PINKY_MCP,Left,0.427986,0.379970,-0.092000,-0.112998,0.235306,-0.092001
6297,6297,HandLandmark.PINKY_PIP,Left,0.400905,0.511519,-0.113890,-0.140079,0.366855,-0.113891
6298,6298,HandLandmark.PINKY_DIP,Left,0.389373,0.591261,-0.121475,-0.151611,0.446597,-0.121475


## 2. Preprocessing
data -> processed_data

- right: 0, left: 1
- landmark: 0~20, total 21

In [3]:
    def preprocessed_data(data, y) :
    # drop unnecessary column: index, landmark
    data = data.drop(['Unnamed: 0', '0'], axis=1)
        
    # mapping hand position: right, left 
    data['1'] = data['1'].replace({'Right': 0, 'Left': 1})
    
    # concating y value at the end of columns 
    data['label'] = y 
    
    # rename columns 
    data = data.rename(columns={"1":"hand_type", "2":"1_x", "3":"1_y", "4":"1_z", "0.1":"1_rx", "1.1":"1_ry", "2.1":"1_rz"})
    
    # add new columns 
    new_cols = []
    for i in range(2, 22) :
        new_cols = new_cols + [str(i)+"_x", str(i)+"_y", str(i)+"_z", str(i)+"_rx", str(i)+"_ry", str(i)+"_rz"]
    data_new = pd.DataFrame(columns=new_cols)
    data = pd.concat([data, data_new])
    
    # insert values at new columns 
    for i in range(0, len(data)-20, 21) :
        for j in range(1, 21) :
            col_list = []
            col_list = [str(j+1)+"_x", str(j+1)+"_y", str(j+1)+"_z", str(j+1)+"_rx", str(j+1)+"_ry", str(j+1)+"_rz"]
            data.loc[i, col_list] = data.loc[i+j, ["1_x", "1_y", "1_z", "1_rx", "1_ry", "1_rz"]].values
            
    # remove values at axis 0 
    remove_cols = []
    for i in range (len(data)) : 
        if i%21!=0 :
            remove_cols.append(i)
    data = data.drop(data.index[remove_cols]).reset_index(drop=True)

    # convert dtypes 
    data[new_cols] = data[new_cols].astype(float)
    
    return data  

In [4]:
processed_dataset = []
for i in range(len(dataset)) :
    processed_data = preprocessed_data(dataset[i], i)
    processed_dataset.append(processed_data)

In [6]:
# check the dataset list 
processed_dataset[1]

,hand_type,1_x,1_y,1_z,1_rx,1_ry,1_rz,label,2_x,2_y,...,20_z,20_rx,20_ry,20_rz,21_x,21_y,21_z,21_rx,21_ry,21_rz
0,0.0,0.503153,0.143476,3.422921e-07,0.0,0.0,0.0,1.0,0.434957,0.265479,...,-0.006647,0.091957,0.284430,-0.006647,0.592520,0.389075,0.000176,0.089367,0.245599,0.000175
1,0.0,0.502994,0.200862,2.832029e-07,0.0,0.0,0.0,1.0,0.439041,0.311611,...,0.008028,0.079900,0.255270,0.008028,0.579888,0.419711,0.013600,0.076894,0.218849,0.013600
2,0.0,0.502033,0.215771,3.003487e-07,0.0,0.0,0.0,1.0,0.444854,0.323350,...,0.010625,0.083254,0.249034,0.010624,0.581322,0.432320,0.016703,0.079289,0.216549,0.016703
3,0.0,0.511145,0.247479,2.584479e-07,0.0,0.0,0.0,1.0,0.455993,0.347283,...,0.007016,0.078261,0.233789,0.007016,0.585713,0.451512,0.012605,0.074568,0.204033,0.012604
4,0.0,0.506148,0.262378,2.510035e-07,0.0,0.0,0.0,1.0,0.452377,0.357633,...,-0.003016,0.080488,0.223370,-0.003016,0.582376,0.457765,0.002514,0.076227,0.195388,0.002514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,0.521973,0.161531,-6.751461e-08,0.0,0.0,0.0,1.0,0.624124,0.261555,...,0.009743,-0.047970,0.291191,0.009743,0.474147,0.413530,0.027447,-0.047827,0.251999,0.027447
296,1.0,0.560034,0.200637,-2.922725e-08,0.0,0.0,0.0,1.0,0.659025,0.300415,...,0.003136,-0.052525,0.285615,0.003136,0.510305,0.446261,0.023548,-0.049729,0.245624,0.023548
297,1.0,0.357869,0.124245,-2.428874e-07,0.0,0.0,0.0,1.0,0.451905,0.232883,...,0.007384,-0.077692,0.290353,0.007384,0.283783,0.371657,0.022099,-0.074086,0.247411,0.022099
298,1.0,0.352533,0.191307,-8.587915e-08,0.0,0.0,0.0,1.0,0.443472,0.296414,...,-0.063270,-0.070835,0.288408,-0.063270,0.282101,0.435142,-0.048966,-0.070432,0.243835,-0.048966


In [7]:
# reshpae as one input data
input_data = processed_dataset[0]
for i in range (1, len(processed_dataset)) :
    input_data = pd.concat([input_data, processed_dataset[i]], ignore_index = True)
input_data

,hand_type,1_x,1_y,1_z,1_rx,1_ry,1_rz,label,2_x,2_y,...,20_z,20_rx,20_ry,20_rz,21_x,21_y,21_z,21_rx,21_ry,21_rz
0,0.0,0.395102,0.732407,-5.277084e-07,0.0,0.0,0.0,0.0,0.318366,0.664055,...,-0.095070,-0.022109,-0.192495,-0.095069,0.395892,0.571559,-0.074054,0.000790,-0.160848,-0.074053
1,0.0,0.447066,0.762395,-5.208548e-07,0.0,0.0,0.0,0.0,0.376279,0.699373,...,-0.079420,-0.000004,-0.190600,-0.079420,0.462811,0.602747,-0.059462,0.015745,-0.159648,-0.059461
2,0.0,0.449753,0.768631,-5.127847e-07,0.0,0.0,0.0,0.0,0.377272,0.703471,...,-0.086723,-0.002741,-0.188313,-0.086723,0.464312,0.612335,-0.066690,0.014559,-0.156296,-0.066689
3,0.0,0.441677,0.821803,-5.137309e-07,0.0,0.0,0.0,0.0,0.363225,0.743152,...,-0.104549,-0.005599,-0.185088,-0.104548,0.458347,0.666532,-0.083679,0.016670,-0.155271,-0.083678
4,0.0,0.463039,0.784468,-5.341338e-07,0.0,0.0,0.0,0.0,0.392481,0.716042,...,-0.091409,0.011701,-0.181360,-0.091409,0.490581,0.632736,-0.070815,0.027542,-0.151731,-0.070815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,1.0,0.552859,0.149594,7.631274e-07,0.0,0.0,0.0,7.0,0.664655,0.236516,...,-0.130920,-0.148239,0.464436,-0.130921,0.400707,0.681821,-0.135373,-0.152152,0.532227,-0.135374
2396,1.0,0.553690,0.147310,7.632809e-07,0.0,0.0,0.0,7.0,0.663573,0.233785,...,-0.124965,-0.148123,0.456344,-0.124966,0.399607,0.671408,-0.129248,-0.154083,0.524098,-0.129249
2397,1.0,0.570058,0.136507,6.770520e-07,0.0,0.0,0.0,7.0,0.681099,0.226392,...,-0.117888,-0.151830,0.464000,-0.117889,0.413520,0.666845,-0.119864,-0.156539,0.530338,-0.119865
2398,1.0,0.536534,0.123091,6.210741e-07,0.0,0.0,0.0,7.0,0.642787,0.211898,...,-0.110878,-0.157269,0.448056,-0.110878,0.371939,0.635951,-0.113966,-0.164595,0.512860,-0.113966


## 3. Data Split

split train and test dataset (8:2)  
split train and valid dataset (8:2)

In [204]:
from sklearn.model_selection import train_test_split

# manage preprocessed data seperately
# -> preprocessed data = procssed_data, input data = input_data 
input_data_y = input_data['label'].copy()
input_data_X = input_data.drop(['label'], axis=1)

# split train/test dataset
tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data_X, 
    input_data_y, 
    test_size = 0.2, DD
    random_state = 42,
    shuffle=True,
    stratify = input_data_y # important option of classification
)

# split valid/train dataset
train_X, valid_X, train_y, valid_y = train_test_split(
    tr_val_X, 
    tr_val_y, 
    test_size = 0.2, 
    random_state = 42,
    shuffle=True,
    stratify = tr_val_y
)

train_data = pd.concat([train_X, train_y], axis=1).reset_index(drop=True)
test_data = pd.concat([test_X, test_y], axis=1).reset_index(drop=True)

##  4. Building Model: Machine Learning 

In [206]:
from lightgbm import LGBMClassifier
from lightgbm import plot_importance 
from xgboost import XGBClassifier
from xgboost import plot_importance
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Learning task parameters
# multi:softmax : multi class classification using softmax 
# multi:softptob : return expected probability for multiple classes using softmax
# mlogloss : multiclass logloss 

model_xgb = XGBClassifier(
    eval_metric='mlogloss')
model_lgbm = LGBMClassifier(
    eval_metric='mlogloss')

## 5. Training

In [207]:
model_xgb.fit(train_X, train_y, eval_set=[(valid_X, valid_y)])

[0]	validation_0-mlogloss:1.06775
[1]	validation_0-mlogloss:0.72702


C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[2]	validation_0-mlogloss:0.51617
[3]	validation_0-mlogloss:0.37394
[4]	validation_0-mlogloss:0.27393
[5]	validation_0-mlogloss:0.20229
[6]	validation_0-mlogloss:0.15044
[7]	validation_0-mlogloss:0.11211
[8]	validation_0-mlogloss:0.08406
[9]	validation_0-mlogloss:0.06344
[10]	validation_0-mlogloss:0.04818
[11]	validation_0-mlogloss:0.03680
[12]	validation_0-mlogloss:0.02828
[13]	validation_0-mlogloss:0.02195
[14]	validation_0-mlogloss:0.01727
[15]	validation_0-mlogloss:0.01371
[16]	validation_0-mlogloss:0.01102
[17]	validation_0-mlogloss:0.00900
[18]	validation_0-mlogloss:0.00743
[19]	validation_0-mlogloss:0.00621
[20]	validation_0-mlogloss:0.00534
[21]	validation_0-mlogloss:0.00460
[22]	validation_0-mlogloss:0.00405
[23]	validation_0-mlogloss:0.00359
[24]	validation_0-mlogloss:0.00321
[25]	validation_0-mlogloss:0.00293
[26]	validation_0-mlogloss:0.00271
[27]	validation_0-mlogloss:0.00254
[28]	validation_0-mlogloss:0.00237
[29]	validation_0-mlogloss:0.00222
[30]	validation_0-mlogloss:0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [208]:
model_lgbm.fit(train_X, train_y, eval_set=[(valid_X, valid_y)])

[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's multi_logloss: 1.45395
[2]	valid_0's multi_logloss: 1.13094
[3]	valid_0's multi_logloss: 0.909599
[4]	valid_0's multi_logloss: 0.740326
[5]	valid_0's multi_logloss: 0.60963
[6]	valid_0's multi_logloss: 0.505648
[7]	valid_0's multi_logloss: 0.421227
[8]	valid_0's multi_logloss: 0.351326
[9]	valid_0's multi_logloss: 0.294028
[10]	valid_0's multi_logloss: 0.246466
[11]	valid_0's multi_logloss: 0.206799
[12]	valid_0's multi_logloss: 0.173355
[13]	valid_0's multi_logloss: 0.145656
[14]	valid_0's multi_logloss: 0.122491
[15]	valid_0's multi_logloss: 0.103199
[16]	valid_0's multi_logloss: 0.0868761
[17]	valid_0's multi_logloss: 0.0733225
[18]	valid_0's multi_logloss: 0.0617943
[19]	valid_0's multi_logloss: 0.0520444
[20]	valid_0's multi_logloss: 0.0438127
[21]	valid_0's multi_logloss: 0.0368227
[22]	valid_0's multi_logloss: 0.0310057
[23]	valid_0's multi_logloss: 0.0261472
[24]	valid_0's multi_logloss: 0.0219906
[25]	valid_0's 

LGBMClassifier(eval_metric='mlogloss')

## 6. Evaluation 

In [1]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

# functions related evaluation 

def show_pred(test_y, pred) :
    #true_y = test_y.to_numpy()
    true_y = np.ravel(true_y)
    
    df_result = pd.DataFrame(list(zip(true_y, pred)), columns=['true_y', 'prediction'])
    return df_result
    
def show_prediction_error(test_y, pred) :
    #true_y = test_y.to_numpy()
    #true_y = np.ravel(true_y)
    error = pred - true_y
    plt.hist(error, bins=25)
    plt.xlabel("Prediction Error")
    _ = plt.ylabel("Count")
    
def feature_importance(model_xgb) : 
    %matplotlib inline
    plt.rcParams['axes.unicode_minus'] = False
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = fm.FontProperties(fname=font_path).get_name()
    rc('font', family=font)

    fig, ax = plt.subplots(figsize=(10,12))
    plot_importance(model_xgb, ax=ax)
    
def graph(pred, test_label) :
    plt.figure(figsize=(16, 9))
    plt.plot(test_label, label = 'actual')
    plt.plot(pred, label = 'prediction')
    plt.legend()
    plt.show()

def classification_report_csv(report) : 
    df = pd.DataFrame(report).transpose()
    df.to_csv('classfication_report.csv', index=False)

C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [220]:
# prob = probability 
pred_xgb_prob = model_xgb.predict_proba(test_X)
pred_xgb = np.argmax(pred_xgb_prob, axis=1)

pred_lgbm_prob = model_lgbm.predict_proba(test_X)
pred_lgbm = np.argmax(pred_lgbm_prob, axis=1)

In [221]:
print(classification_report(test_y, pred_xgb))

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99        62
         1.0       1.00      1.00      1.00        62
         2.0       1.00      1.00      1.00        60
         3.0       1.00      1.00      1.00        60
         4.0       1.00      1.00      1.00        60
         5.0       0.98      1.00      0.99        60
         6.0       1.00      1.00      1.00        60
         7.0       1.00      1.00      1.00        60

    accuracy                           1.00       484
   macro avg       1.00      1.00      1.00       484
weighted avg       1.00      1.00      1.00       484



In [222]:
print(classification_report(test_y, pred_lgbm))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        62
         1.0       1.00      1.00      1.00        62
         2.0       0.98      1.00      0.99        60
         3.0       1.00      1.00      1.00        60
         4.0       1.00      0.98      0.99        60
         5.0       1.00      1.00      1.00        60
         6.0       1.00      1.00      1.00        60
         7.0       1.00      1.00      1.00        60

    accuracy                           1.00       484
   macro avg       1.00      1.00      1.00       484
weighted avg       1.00      1.00      1.00       484



## 7. Exporting

In [223]:
from pickle import dump
import pickle

In [224]:
with open('model_xgb.h5', 'wb') as file:  
    pickle.dump(model_xgb, file)

In [225]:
with open('model_lgbm.h5', 'wb') as file:  
    pickle.dump(model_lgbm, file)

## AutoML

From here, we just experiment using autoML library, so this is not a perfect code

In [ ]:
from pycaret.classification import *
from time import time

In [ ]:
# Setup Environment  
clf = setup(data = train_data, 
            test_data = test_data,
            target = 'label',
            data_split_stratify = True,
            fold_shuffle=True,
            use_gpu = True,
            silent = True,
            session_id = 6315)

In [13]:
# et, rf, 
top5 = compare_models(sort='Accuracy', n_select=5, fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8659,0.9839,0.8660,0.8660,0.8658,0.8467,0.8468,5.6020
rf,Random Forest Classifier,0.8485,0.9823,0.8487,0.8484,0.8484,0.8269,0.8269,3.6640
catboost,CatBoost Classifier,0.8395,0.9853,0.8395,0.8396,0.8394,0.8166,0.8166,17.2360
lightgbm,Light Gradient Boosting Machine,0.8281,0.9833,0.8280,0.8287,0.8280,0.8035,0.8036,1.6700
knn,K Neighbors Classifier,0.7556,0.9467,0.7552,0.7600,0.7559,0.7207,0.7212,4.8160
dt,Decision Tree Classifier,0.7416,0.8523,0.7416,0.7420,0.7417,0.7047,0.7047,0.3920
gbc,Gradient Boosting Classifier,0.7078,0.9524,0.7073,0.7112,0.7075,0.6660,0.6665,49.5000
lr,Logistic Regression,0.5275,0.8802,0.5262,0.5261,0.5248,0.4600,0.4605,6.8820
lda,Linear Discriminant Analysis,0.5008,0.8722,0.4998,0.5111,0.5014,0.4295,0.4306,0.1420
ridge,Ridge Classifier,0.4618,0.0000,0.4599,0.4536,0.4458,0.3846,0.3875,0.0540
